In [1]:
import pandas as pd
import numpy as np
from env import get_db_url



In [4]:
def getzillow():
    sql_query = '''
    select * from properties_2017
    join predictions_2017 using (parcelid)
    where transactiondate < '2018'
    and propertylandusetypeid = 261;
    '''
    
    zdf = pd.read_sql(sql_query, get_db_url('zillow'))
    zdf = zdf.drop(columns='id')
    return zdf

In [5]:
zdf = getzillow()

In [6]:
#zdf.head()

In [7]:
#zdf.info()

In [8]:
#zdf.describe(include='all')

In [10]:
print(f'rows: {zdf.shape[0]}')
print(f'co1umns: {zdf.shape[1]}')

rows: 52441
co1umns: 60


In [11]:
#zdf.columns

In [12]:
#for column in zdf.columns:
#    print(column)
#    print(zdf[column].value_counts())
#    print('\n ------------------------- \n')

In [14]:
#for column in zdf.columns:
#    print(f'{column}\n{zdf[column].value_counts()} \n\n ------------------------- \n')


In [16]:
#zdf.isnull()


In [18]:
#zdf.isnull().sum()


In [19]:
zdf.shape[0]

52441

In [21]:
#(zdf.isnull().sum()/zdf.shape[0]*100).sort_values(ascending=False)


In [25]:
nullpercents = pd.DataFrame({'num_rows_missing': zdf.isnull().sum(), 'percent_rows_missing':zdf.isnull().sum()/zdf.shape[0]*100})

In [ ]:
#nullpercents

In [24]:
nullpercents.loc['logerror']

num_rows_missing        0.0
percent_rows_missing    0.0
Name: logerror, dtype: float64

In [26]:
def nullbycol(df):
    num_missing = zdf.isnull().sum()
    rows = zdf.shape[0]
    percent_missing = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': percent_missing})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [28]:
#nullbycol(zdf)

In [30]:
#zdf.isnull().sum(axis=1)

In [33]:
#zdf.isnull().sum(axis=1) / zdf.shape[1] * 100


In [36]:
null_row_df = pd.DataFrame({'num_cols_missing': zdf.isnull().sum(axis=1),
             'percent_cols_missing': zdf.isnull().sum(axis=1)/zdf.shape[1]*100})

In [37]:
#null_row_df

In [39]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    percent_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing':num_missing, 'percent_cols_missing':percent_miss})
    rows_missing = df.merge(rows_missing, left_index=True, right_index=True)[['parcelid','num_cols_missing','percent_cols_missing']]
    return rows_missing.sort_values(by='num_cols_missing',ascending=False)

In [41]:
#nulls_by_row(zdf)

In [42]:

# def summarize(df):
#     '''
#     summarize will take in a single argument (a pandas dataframe) 
#     and output to console various statistics on said dataframe, including:
#     # .head()
#     # .info()
#     # .describe()
#     # .value_counts()
#     # observation of nulls in the dataframe
#     '''
#     print('SUMMARY REPORT')
#     print('=====================================================\n\n')
#     print('Dataframe head: ')
#     print(df.head(3))
#     print('=====================================================\n\n')
#     print('Dataframe info: ')
#     print(df.info())
#     print('=====================================================\n\n')
#     print('Dataframe Description: ')
#     print(df.describe())
#     num_cols = [col for col in df.columns if df[col].dtype != 'O']
#     cat_cols = [col for col in df.columns if col not in num_cols]
#     print('=====================================================')
#     print('DataFrame value counts: ')
#     for col in df.columns:
#         if col in cat_cols:
#             print(df[col].value_counts(), '\n')
#         else:
#             print(df[col].value_counts(bins=10, sort=False), '\n')
#     print('=====================================================')
#     print('nulls in dataframe by column: ')
#     print(nulls_by_col(df))
#     print('=====================================================')
#     print('nulls in dataframe by row: ')
#     print(nulls_by_row(df))
#     print('=====================================================')


In [43]:
def remove_columns(df, cols_to_remove):
    df = df.drop(columns=cols_to_remove)
    return df

In [45]:
def handle_missing_values(df, prop_required_columns=0.5, prop_required_rows=0.75):
    column_threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=column_threshold)
    row_threshold = int(round(prop_required_rows * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=row_threshold)
    return df

In [46]:
def data_prep(df, col_to_remove=[], prop_required_columns=0.5, prop_required_rows=0.75):
    df = remove_columns(df, col_to_remove)
    df = handle_missing_values(df, prop_required_columns, prop_required_rows)
    return df

Handling Outliers
Note your use case
z-score: appropriate for normal data (normally distributed)
Tukey IQR method: not contingent on normality
Tukey Interquartile Range (IQR) Method:
Calculate IQR
Get Q3 and Q1
Get difference (q3-q1)
Establish "fences":
Standard inner fence: k = 1.5
Standard outer fence: k = 3.0
Upper bound: q3 + k * IQR
Lower bound: q1 - k * IQR

In [48]:
def get_upper_outliers(s, k=1.5):
    '''given a series and cutoff K, returned upper outliers'''
    q1, q3 = s.quantile([.25,0.75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x-upper_bound,0]))

In [54]:
def add_upper_outlier_columns(df, k=1.5):
    '''add column iwt suffix _outliers'''
    for col in df.select_dtypes('number'):
        df[col + '_outliers_upper'] = get_upper_outliers(df[col], k)
    return df

In [55]:
df = add_upper_outlier_columns(zdf)

In [57]:
#df

In [59]:
# outlier_cols = [col for col in df.columns if col.endswith('_outliers_upper')]
# for col in outlier_cols:
#     print(col, ': ')
#     subset = df[col][df[col] > 0]
#     print(f'Number of Observations Above Upper Bound: {subset.count()}', '\n')
#     print(subset.describe())
#     print('------', '\n')